In [2]:
# domo
import domojupyter as domo 

#pandas
import pandas as pd
#from pandas import json_normalize

#!pip install requests
#import requests
#import time
#import json
import re

#!pip install authlib
#import authlib
#from authlib.jose import jwt

#import os
#os.getcwd()

# date manipulations
#from datetime import datetime
#import calendar as calendar
#!pip install holidays
#import holidays

#import matplotlib.pyplot as plt
#import numpy as np

# Tutorial: https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment?text=1st+time+using+the+app...+everything+seemed+to+be+working+fine.+when+the+app+said+my+order+was+ready%2C+i+drove+to+the+restaurant.+when+i+got+there%2C+they+said+they+were+taking+care+of+the+drive+through+orders+1st.+well%2C+thats+fine%2C+but+the+app+said+my+order+was+ready%3F+ready+is+ready...+right%3F%3F%3F+why+did+i+have+to+wait%3F+i+still+had+to+wait+over+20+minutes+to+get+the+order+filled.+i+don%27t+think+i+will+be+using+this+app+again.+there+is+no+point+if+the+local+store+is+this+inept.+%F0%9F%91%8E%F0%9F%91%8E%F0%9F%91%8E
#!pip install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html
!pip install tensorflow==2.9
!pip install keras==2.9
#!pip install keras
#!pip install tensorflow
!pip install transformers
!pip install torch

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import AutoModel, TFAutoModel
from transformers import pipeline
#import numpy as np
!pip install scipy
#!pip install nltk
import scipy
from scipy.special import softmax
import urllib.request

#import csv
#from nltk.tokenize import tokenize


/home/domo/.conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-19 11:32:18.322958: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-19 11:32:18.619477: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-19 11:32:18.619510: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# select all columns from the Domo Dataset
input = domo.read_dataframe('Eng vs Wales Tweets Clean', query='SELECT * FROM table')

# pass the three columns erequired for analysis to a dataframe
df = input[['data/id', 'data/created_at', 'data/text']]

# Check Processing
df.tail(10)

,data/id,data/created_at,data/text
9750,1597670604031012864,2022-11-29 19:15:29,Benni McCarthy watching that Rashford chance \...
9751,1597670606967050240,2022-11-29 19:15:30,Looking forward to all the BBC sports punters ...
9752,1597670604513357824,2022-11-29 19:15:30,I played in an over 35s game at the weekend th...
9753,1597670607638106112,2022-11-29 19:15:30,'@mrdanwalker Yes...no..maybe...not sure...lol
9754,1597670611383652352,2022-11-29 19:15:31,Jesus. \r\n\r\nAnother Tory who stays silent a...
9755,1597670609982750720,2022-11-29 19:15:31,C.K Akonnor be co commentor for Wales vrs Engl...
9756,1597670612956160000,2022-11-29 19:15:32,'@RRPNP Up for whales 😅😅😅
9757,1597670620938260480,2022-11-29 19:15:33,You could easily fall asleep watching this Eng...
9758,1597670617427324928,2022-11-29 19:15:33,What a load of rubbish #ENGWAL
9759,1597670625212239872,2022-11-29 19:15:34,When the question is Ronnie Whelan or Jermain ...


In [5]:
df = df.rename(columns={'data/id': 'Tweet ID', 
                      'data/created_at': 'Date', 
                      'data/text': 'Text'})

# convert the data type of the inquiries from float to string & Set test to lower case
df['Text'] = df['Text'].astype(str) 
df['Text'] = df['Text'].str.lower() 

#Check Processing
df.head(10)



,Tweet ID,Date,Text
0,1597644704610672644,2022-11-29 17:32:35,these guys are wasted lol https://t.co/g1cwiorwce
1,1597644696318541825,2022-11-29 17:32:33,'@cupstakes 0-0 draw
2,1597644726655537153,2022-11-29 17:32:40,a spot in the last 16 is pretty much a certain...
3,1597644710486880257,2022-11-29 17:32:36,'@footballs0cial when was the last time wales ...
4,1597644695177678848,2022-11-29 17:32:32,'@juliahb1 @garethsouthgate stiff as a board 🤣
5,1597644657470877696,2022-11-29 17:32:23,george versus the dragon. we all know what hap...
6,1597644695110578176,2022-11-29 17:32:32,#engwal i’ve arrived home to this 🤬. it’s goin...
7,1597644757089386496,2022-11-29 17:32:47,'@gtomegaracing wales 🏴󠁧󠁢󠁷󠁬󠁳󠁿 0-1 🏴󠁧󠁢󠁥󠁮󠁧󠁿 engl...
8,1597644783606190080,2022-11-29 17:32:53,convinced that whoever is managing europe bere...
9,1597644786462527489,2022-11-29 17:32:54,'@chelstransfer sterling to score first when h...


In [6]:

# Remove links from tweet text column passes values to a new column (Not 100% sure this is catching all links)
df['text_no_link'] = df['Text'].apply(lambda x: re.sub(r'http\S+', '', x))

# Drop original text column
df.drop('Text', axis=1, inplace=True)

# Rename clean text Column
df = df.rename(columns={'text_no_link': 'Text'})

#df.shape
df.tail(20)


,Tweet ID,Date,Text
9740,1597670579800510464,2022-11-29 19:15:24,is tom kerridge commentating on this match? #w...
9741,1597670581083983872,2022-11-29 19:15:24,struggling to get to a tv atm.\r\n\r\nhow’s th...
9742,1597670584598421504,2022-11-29 19:15:25,'@england if you need a bet based on bet build...
9743,1597670591792050178,2022-11-29 19:15:27,"please don’t turn up danny, save it for leices..."
9744,1597670593406849024,2022-11-29 19:15:27,i genuinely do support footballers taking the ...
9745,1597670597282369537,2022-11-29 19:15:28,rashford goes to the same school of shooting a...
9746,1597670597521117185,2022-11-29 19:15:28,we are so unbelievably negative. some of the b...
9747,1597670597370478592,2022-11-29 19:15:28,the heat of #irnusa overcame the #waleng \r\n\...
9748,1597670602541740032,2022-11-29 19:15:29,"why dont @englandfootball ever pass, move into..."
9749,1597670601862549505,2022-11-29 19:15:29,should marcus rashford should have scored from...


In [7]:
# Import model
classifier = pipeline("sentiment-analysis", model="j-hartmann/emotion-english-distilroberta-base", truncation=True)

In [8]:
def classifier_emotion(text):
  classifier_results = classifier(text)
  return classifier_results[0]['label']
    
def classifier_score(text):
 classifier_results = classifier(text)
 return classifier_results[0]['score']



In [9]:
df['Emotion'] = df['Text'].apply(lambda x: classifier_emotion(x))
df['Score'] = df['Text'].apply(lambda x: classifier_score(x))

#convert the emotion column to title case
df['Emotion'] = df['Emotion'].str.title()
df

,Tweet ID,Date,Text,Emotion,Score
0,1597644704610672644,2022-11-29 17:32:35,these guys are wasted lol,Sadness,0.322998
1,1597644696318541825,2022-11-29 17:32:33,'@cupstakes 0-0 draw,Sadness,0.380771
2,1597644726655537153,2022-11-29 17:32:40,a spot in the last 16 is pretty much a certain...,Fear,0.751376
3,1597644710486880257,2022-11-29 17:32:36,'@footballs0cial when was the last time wales ...,Surprise,0.597759
4,1597644695177678848,2022-11-29 17:32:32,'@juliahb1 @garethsouthgate stiff as a board 🤣,Fear,0.572718
...,...,...,...,...,...
9755,1597670609982750720,2022-11-29 19:15:31,c.k akonnor be co commentor for wales vrs engl...,Neutral,0.462914
9756,1597670612956160000,2022-11-29 19:15:32,'@rrpnp up for whales 😅😅😅,Neutral,0.637170
9757,1597670620938260480,2022-11-29 19:15:33,you could easily fall asleep watching this eng...,Sadness,0.619128
9758,1597670617427324928,2022-11-29 19:15:33,what a load of rubbish #engwal,Anger,0.925219


In [ ]:
domo.write_dataframe(df, 'Eng vs Wales Tweets Processed')